# _Flower Recognition - Classification Project_

<img src='../data/flower.webp'>

_Bu proje, **Python ve makine öğrenmesi** kullanarak çiçek görsellerini tanımaya odaklanmaktadır. Amaç, bir çiçek fotoğrafını renk ve kenar özelliklerine göre **beş sınıftan birine** sınıflandırmaktır._

## _Veri Seti_
- _Toplam **4242 çiçek görseli** içeriyor._
- _Görseller **Flickr, Google Görseller ve Yandex Görseller**den toplandı._
- _**5 sınıf mevcut:**_
  - _Papatya (Chamomile)_
  - _Lale (Tulip)_
  - _Gül (Rose)_
  - _Ayçiçeği (Sunflower)_
  - _Karahindiba (Dandelion)_
- _Görseller düşük çözünürlüklü (~320x240) ve farklı boyutlarda._

## _Adımlar_
1. _**Veri Hazırlığı**_
   - _Görseller ve etiketler yüklenir._
   - _Gerekirse görseller standart boyuta getirilir._
   - _Görseller ML modellerinin anlayacağı şekilde diziye çevrilir._ <br><br>

2. _**Özellik Çıkarımı**_
   - _Görsellerin renk ve kenar özellikleri kullanılır._
   - _Örnekler: HOG özellikleri, renk histogramları veya doğrudan piksel değerleri._ <br><br>

3. _**Eğitim/Test Bölme**_
   - _Veri seti eğitim ve test olarak ayrılır._ <br><br>

4. _**Model Eğitimi**_
   - _Kullanılabilecek modeller:_
     - _Random Forest, SVM, KNN veya_
     - _Derin öğrenme için Convolutional Neural Network (CNN)_
   - _Model eğitim verisi ile eğitilir._ <br><br>

5. _**Model Değerlendirme**_
   - _Doğruluk (accuracy), precision, recall gibi metrikler ile performans değerlendirilir._
   - _Yanlış sınıflandırılan görseller gözlemlenir._ <br><br>

6. _**Tahmin**_
   - _Eğitilen model kaydedilir._
   - _Yeni bir görselin sınıfı tahmin edilir._ <br><br>

7. _**Streamlit Arayüzü**_
   - _Kullanıcı çiçek görseli yükleyebilir ve model tahminini görebilir._ 

### _İmport_

In [1]:
# OpenCV kütüphanesini import ediyoruz, görüntü işleme ve video/frame işlemleri için kullanılır
import cv2

# Pandas kütüphanesini import ediyoruz, veri analizi ve tablo şeklinde veri yönetimi için kullanılır
import pandas as pd

# NumPy kütüphanesini import ediyoruz, sayısal hesaplamalar ve matris işlemleri için kullanılır
import numpy as np

# scikit-learn kütüphanesinden train_test_split fonksiyonunu import ediyoruz,
# veriyi eğitim ve test setlerine ayırmak için kullanılır
from sklearn.model_selection import train_test_split

# Keras kütüphanesinden gerekli modülleri import ediyoruz
# Sequential: katman katman model oluşturmak için
# Conv2D: 2D evrişim katmanı (görüntülerde özellik çıkarımı için)
# Dense: tam bağlantılı katman (sınıflandırma veya regresyon için)
# Flatten: çok boyutlu veriyi tek boyuta indirger
# Input: modelin giriş katmanı
# MaxPooling2D: evrişim katmanından sonra boyut küçültmek için max pooling uygular
# Dropout: aşırı öğrenmeyi (overfitting) önlemek için rastgele nöronları kapatır
# BatchNormalization: modelin daha hızlı ve stabil öğrenmesini sağlar
# Reshape: verinin şeklini değiştirmek için
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

# İşletim sistemi ile ilgili işlemler yapmak için os modülünü import ediyoruz
# Örneğin dosya/dizin kontrolleri, dosya yolları oluşturma gibi
import os

In [2]:
# Python'da uyarı mesajlarını yönetmek için warnings modülünü import ediyoruz
import warnings

# Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz
# Bu sayede ekranda gereksiz uyarılar görünmez ve kod çıktısı daha temiz olur
warnings.filterwarnings('ignore')

### _Eda_

In [3]:
# Sınıf etiketlerini bir liste halinde tanımlıyoruz
# 'Cancer' -> Kanserli örnekler
# 'Non_Cancer' -> Kanserli olmayan örnekler
labels = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Bu klasör içinde 'Cancer' ve 'Non_Cancer' alt klasörleri olabilir
img_path = '../data/flowers/'

In [4]:
#ls

In [5]:
# Boş bir liste oluşturuyoruz; görüntülerin dosya yollarını buraya ekleyeceğiz
img_list = []

# Boş bir liste oluşturuyoruz; her görüntüye karşılık gelen etiketi buraya ekleyeceğiz
label_list = []

# labels listesindeki her sınıf (ör. 'Cancer' ve 'Non_Cancer') için döngü
for label in labels:
    
    # Her sınıfın klasöründeki tüm dosyaları listele
    for img_file in os.listdir(img_path + label):
        
        # Görüntü dosyasının tam yolunu oluştur ve img_list'e ekle
        img_list.append(img_path + label + "/" + img_file)
        
        # Görüntünün ait olduğu sınıf etiketini label_list'e ekle
        label_list.append(label)

In [6]:
# img_list ve label_list listelerini kullanarak bir DataFrame oluşturuyoruz
# DataFrame, tablo şeklinde veri tutmamızı sağlar (sütunlar: 'img' ve 'label')
df = pd.DataFrame({'img': img_list,  # 'img' sütunu: görüntü dosya yolları
                   'label': label_list})  # 'label' sütunu: her görüntünün sınıf etiketi

In [7]:
df

,img,label
0,../data/flowers/daisy/100080576_f52e8ee070_n.jpg,daisy
1,../data/flowers/daisy/10140303196_b88d3d6cec.jpg,daisy
2,../data/flowers/daisy/10172379554_b296050f82_n...,daisy
3,../data/flowers/daisy/10172567486_2748826a8b.jpg,daisy
4,../data/flowers/daisy/10172636503_21bededa75_n...,daisy
...,...,...
4312,../data/flowers/tulip/9831362123_5aac525a99_n.jpg,tulip
4313,../data/flowers/tulip/9870557734_88eb3b9e3b_n.jpg,tulip
4314,../data/flowers/tulip/9947374414_fdf1d0861c_n.jpg,tulip
4315,../data/flowers/tulip/9947385346_3a8cacea02_n.jpg,tulip


In [9]:
# Sınıf isimlerini sayısal değerlere eşleyecek bir sözlük oluşturuyoruz
# Çiçek isimlerini sayılara eşleme
d = {
    'daisy': 0,
    'dandelion': 1,
    'rose': 2,
    'sunflower': 3,
    'tulip': 4
}

# DataFrame'deki 'label' sütununu sayısal değerlere çeviriyoruz
# map fonksiyonu, her etiketi sözlükteki karşılığı ile değiştirir
df['label_encoded'] = df['label'].map(d)

In [10]:
df

#Oluşturduğumuz dataframein tamamını görüntüleyebiliriz.

,img,label,label_encoded
0,../data/flowers/daisy/100080576_f52e8ee070_n.jpg,daisy,0
1,../data/flowers/daisy/10140303196_b88d3d6cec.jpg,daisy,0
2,../data/flowers/daisy/10172379554_b296050f82_n...,daisy,0
3,../data/flowers/daisy/10172567486_2748826a8b.jpg,daisy,0
4,../data/flowers/daisy/10172636503_21bededa75_n...,daisy,0
...,...,...,...
4312,../data/flowers/tulip/9831362123_5aac525a99_n.jpg,tulip,4
4313,../data/flowers/tulip/9870557734_88eb3b9e3b_n.jpg,tulip,4
4314,../data/flowers/tulip/9947374414_fdf1d0861c_n.jpg,tulip,4
4315,../data/flowers/tulip/9947385346_3a8cacea02_n.jpg,tulip,4


### _Classification_

In [11]:
# Boş bir liste oluşturuyoruz; işlenmiş görüntüleri buraya ekleyeceğiz
x = []  

# DataFrame'deki tüm görüntü dosya yolları üzerinde döngü
for img in df['img']:
    
    # Görüntüyü OpenCV ile oku
    img = cv2.imread(str(img))
    
    # Görüntüyü 170x170 boyutuna yeniden boyutlandır
    # Not: ResNet transfer learning kullanacağımız için 170x170 girdik
    # Ama başka projelerde yeterli RAM varsa istediğimiz başka boyutu da kullanabiliriz
    img = cv2.resize(img, (170, 170))
    
    # Piksel değerlerini normalize et (0-1 aralığına getir)
    img = img / 255.0
    
    # İşlenmiş görüntüyü listeye ekle
    x.append(img)

In [12]:
# x listesini NumPy dizisine çeviriyoruz
# Çünkü makine öğrenmesi ve derin öğrenme kütüphaneleri genellikle NumPy dizileri ile çalışır
x = np.array(x)  # Artık x, modelin anlayacağı şekilde çok boyutlu bir dizi (array)

In [13]:
x.shape

(4317, 170, 170, 3)

In [14]:
# DataFrame'den sadece 'label_encoded' sütununu seçiyoruz
# Bu sütun, her görüntünün sayısal sınıf etiketini (0 veya 1) içeriyor
y = df[['label_encoded']]

In [15]:
# Veriyi eğitim ve test setlerine ayırıyoruz
# x → görüntü verileri (girdi)
# y → etiketler (çıktı)
# test_size=0.20 → verinin %20'si test seti, %80'i eğitim seti olacak
# random_state=42 → rastgele bölme işleminin tekrarlanabilir olmasını sağlar
#                  42 sadece farklı bir sabit sayı, aynı sayıyı her kullanışta aynı bölme elde edilir
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [16]:
# Eğitim etiketlerini NumPy dizisine çeviriyoruz ve veri tipini int32 olarak belirliyoruz
y_train = np.array(y_train, dtype=np.int32)

# Test etiketlerini NumPy dizisine çeviriyoruz ve veri tipini int32 olarak belirliyoruz
y_test = np.array(y_test, dtype=np.int32)

In [19]:
# Sequential model oluşturuyoruz, katman katman model ekleyebileceğiz
model = Sequential()

# Modelin giriş katmanını tanımlıyoruz
# Girdi boyutu: 170x170 piksel, 3 renk kanalı (RGB)
model.add(Input(shape=(170, 170, 3)))

# 1. Convolution (evrişim) katmanı
# 32 filtre, 3x3 boyutunda, aktivasyon fonksiyonu ReLU
# Görüntüden özellikler çıkarır
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

# 1. MaxPooling katmanı
# 2x2 boyutunda, uzaysal boyutları küçültür ve hesaplamayı azaltır
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2. Convolution katmanı
# 64 filtre, 3x3 boyutunda, aktivasyon fonksiyonu ReLU
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# 2. MaxPooling katmanı
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten katmanı
# Çok boyutlu veriyi tek boyuta indirger, Dense katmanına girdi olarak verir
model.add(Flatten())

# Tam bağlantılı (Dense) katman
# 128 nöron, ReLU aktivasyonu
# Görüntüden çıkarılan özellikleri birleştirir ve öğrenir
model.add(Dense(128, activation='relu'))

# Çıkış katmanı
# 1 nöron, sigmoid aktivasyonu
# Binary classification için 0-1 arasında tahmin verir
model.add(Dense(5, activation='softmax'))

# Modeli derliyoruz
# optimizer='adam' → ağırlıkları güncellemek için Adam optimizasyonu
# loss='binary_crossentropy' → binary sınıflandırma için uygun kayıp fonksiyonu
# metrics=['accuracy'] → eğitimi izlerken doğruluk metriğini hesaplar
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Modeli eğitim verisiyle eğitiyoruz
history = model.fit(
    x_train,              # Girdi verileri (görüntüler)
    y_train,              # Hedef etiketler (0 veya 1)
    validation_data=(x_test, y_test),  # Her epoch sonunda test verisi ile doğrulama
    epochs=20,            # Modelin tüm eğitim verisi üzerinden 20 kez geçmesi
    verbose=1             # Eğitim sırasında ilerleme çubuğunu göster
)

Epoch 1/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 168ms/step - accuracy: 0.3698 - loss: 1.7534 - val_accuracy: 0.4965 - val_loss: 1.2835
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 162ms/step - accuracy: 0.6015 - loss: 1.0340 - val_accuracy: 0.5671 - val_loss: 1.1288
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 164ms/step - accuracy: 0.7396 - loss: 0.7053 - val_accuracy: 0.5938 - val_loss: 1.1506
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 165ms/step - accuracy: 0.8810 - loss: 0.3749 - val_accuracy: 0.5949 - val_loss: 1.4030
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 17s 160ms/step - accuracy: 0.9545 - loss: 0.1655 - val_accuracy: 0.5451 - val_loss: 1.9236
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 164ms/step - accuracy: 0.9763 - loss: 0.0994 - val_accuracy: 0.5683 - val_loss: 2.0146
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 167ms/step - accuracy: 0.9806 - loss: 0.0763 - val_accuracy: 0.5532 - val_loss: 2.2086
Epoch 8/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 163ms/step - accuracy: 0.9904 - loss: 0

### _Save the Model_

In [21]:
model.save('flower_recognition_model.keras') # huggine kaydetmek istiyorsam .h5 ile kaydetmem lazım

### _Transfer Learning_

_Akıllı insan aklını kullanandır. Daha da akıllı insan başkalarınında aklını kullanandır_

#### _İmport_

In [22]:
# Keras'ın hazır transfer learning modellerini import ediyoruz
# VGG16 ve ResNet50, önceden ImageNet veri setinde eğitilmiş derin CNN modelleridir
from tensorflow.keras.applications import VGG16, ResNet50

# Görüntü verilerini işlemek ve artırmak (augmentation) için ImageDataGenerator sınıfını import ediyoruz
# Örneğin: döndürme, yakınlaştırma, kaydırma gibi işlemlerle eğitim verisini zenginleştirmek için kullanılır
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### _İmport Another Model - Classification_

In [30]:
# Ana veri klasörünün yolunu belirliyoruz
data_dir = '../data/flowers'

# Modelin girdi boyutunu belirliyoruz (224x224 piksel)
img_width, img_heigth = 224, 224

# Eğitim verisi için ImageDataGenerator oluşturuyoruz
# rescale=1/255 → piksel değerlerini 0-1 aralığına normalize ediyor
# validation_split=0.20 → verinin %20'sini doğrulama için ayırıyor
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.20)

# Eğitim verilerini klasörden okuyup artırma ve hazırlama
train_datagenerator = train_datagen.flow_from_directory(
    directory=data_dir,              # Ana veri klasörü
    target_size=(img_width,img_heigth),  # Görüntüleri 224x224 boyutuna getir
    class_mode='categorical',             # categorical sınıflandırma
    subset='training'                # Eğitim verisi olarak ayır
)

# Test/verifikasyon verilerini hazırlıyoruz
test_datagen = ImageDataGenerator(rescale=1/255)

# Doğrulama verilerini klasörden alıyoruz
test_datagenerator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_width,img_heigth),
    class_mode='categorical',
    subset='validation'              # Doğrulama seti
)

# Önceden eğitilmiş VGG16 modelini yükle
# weights='imagenet' → ImageNet üzerinde önceden eğitilmiş ağırlıklar
# include_top=False → son sınıflandırma katmanını dahil etme (kendi katmanlarımızı ekleyeceğiz)
base_model = VGG16(weights='imagenet', input_shape=(img_width,img_heigth,3), include_top=False)

# Yeni bir Sequential model oluşturuyoruz
model = Sequential()
model.add(base_model)  # Önceden eğitilmiş VGG16 tabanını ekliyoruz

# VGG16 tabanındaki tüm katmanları donduruyoruz, yani eğitim sırasında güncellenmeyecek
for layer in base_model.layers:
    layer.trainable = False

# Kendi üst katmanlarımızı ekliyoruz
model.add(Flatten())            # Çok boyutlu özellik haritasını tek boyuta çevir
model.add(Dense(1024, activation='relu'))  # Fully connected katman, öğrenilen özellikleri birleştir
model.add(Dense(5, activation='softmax'))  # Çıkış katmanı, softmax sınıflandırma

# Modeli derliyoruz
model.compile(
    optimizer='adam',                 # Ağırlıkları güncellemek için Adam optimizasyonu
    loss='categorical_crossentropy',       # categorical_crossentropy sınıflandırma kayıp fonksiyonu
    metrics=['accuracy']              # Eğitim ve doğrulama sırasında doğruluk metriğini takip et
)

# Modeli eğitim verisiyle eğitiyoruz ve doğrulama verisiyle test ediyoruz
# epochs=10 → veri üzerinden 10 kez geçiyoruz
model.fit(
    train_datagenerator,
    epochs=5,
    validation_data=test_datagenerator
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.
Epoch 1/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.6468 - loss: 2.0861 - val_accuracy: 0.8000 - val_loss: 0.5370
Epoch 2/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.8776 - loss: 0.3581 - val_accuracy: 0.7895 - val_loss: 0.5753
Epoch 3/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.9387 - loss: 0.1808 - val_accuracy: 0.8174 - val_loss: 0.5248
Epoch 4/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.9876 - loss: 0.0692 - val_accuracy: 0.8081 - val_loss: 0.5491
Epoch 5/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.9980 - loss: 0.0296 - val_accuracy: 0.8151 - val_loss: 0.5577


#### _Save the Transfer Learning Model_

In [31]:
model.save('flower_recognition_transfer_learning.keras')